# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [399]:
import pandas as pd
import requests
from statistics import mode

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [400]:
# 2) seu código aqui
sinasc_1 = pd.DataFrame(sinasc)
sinasc_1.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [401]:
# 3) seu código aqui
sinasc_reduzida = sinasc_1[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_reduzida.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [402]:
# 4) seu código aqui
sinasc_reduzida.APGAR5.isna().sum()
sinasc_reduzida[sinasc_reduzida.APGAR5.isna()] #todas missing 
sinasc_limpa = sinasc_reduzida[~sinasc_reduzida.APGAR5.isna()] #todas que nao tem missing 

print (f'Apos a limpeza ficaram {sinasc_limpa.shape} e teve um total de {sinasc_limpa.APGAR5.isna().sum()} \
linhas APGAR5 com dados faltantes')


Apos a limpeza ficaram (26925, 9) e teve um total de 0 linhas APGAR5 com dados faltantes


In [403]:
print (sinasc_limpa.shape)
sinasc_limpa.head()

(26925, 9)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [404]:
# 5) seu código aqui
sinasc2 = sinasc_limpa.copy()

print (f'Os dados "ESTCIVMAE" possui um total de {sinasc_limpa.ESTCIVMAE.isna().sum()} \
dados faltantes e "CONSULTAS" tambêm possui {sinasc_limpa.CONSULTAS.isna().sum()}.')

sinasc2['ESTCIVMAE'] = sinasc2['ESTCIVMAE'].fillna(9)
sinasc2['CONSULTAS'] = sinasc2['CONSULTAS'].fillna(9)

print (f'E apos o tratamento possui respectivamente {sinasc2.ESTCIVMAE.isna().sum()} \
e {sinasc2.CONSULTAS.isna().sum()}.')




Os dados "ESTCIVMAE" possui um total de 315 dados faltantes e "CONSULTAS" tambêm possui 0.
E apos o tratamento possui respectivamente 0 e 0.


In [406]:
# 6) Seu código aqui
sinasc2['QTDFILVIVO'] = sinasc2['QTDFILVIVO'].fillna(0)

print (f'Na coluna Quantidade de filhos possui {sinasc2.QTDFILVIVO.isna().sum()} com valores sem respostas "NaN"')

Na coluna Quantidade de filhos possui 0 com valores sem respostas "NaN"


In [407]:
# 7) seu código aqui
dados = sinasc2[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

for coluna in dados:
    print (f'O Dado {coluna} possui {sinasc2[coluna].isna().sum()} dados faltantes')

O Dado LOCNASC possui 0 dados faltantes
O Dado IDADEMAE possui 0 dados faltantes
O Dado ESTCIVMAE possui 0 dados faltantes
O Dado ESCMAE possui 310 dados faltantes
O Dado QTDFILVIVO possui 0 dados faltantes
O Dado GESTACAO possui 1216 dados faltantes
O Dado GRAVIDEZ possui 76 dados faltantes
O Dado CONSULTAS possui 0 dados faltantes
O Dado APGAR5 possui 0 dados faltantes


In [408]:
# 7) seu código aqui
sinasc2.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [409]:
sinasc2['GRAVIDEZ']= sinasc2['GRAVIDEZ'].map({'Única': 1,'Dupla': 2, 'Tripla e mais' : 3, 'Ignorado' : 9 })

In [410]:
moda_gravidez = mode(sinasc2['GRAVIDEZ'])
sinasc2['GRAVIDEZ'] = sinasc2['GRAVIDEZ'].fillna(moda_gravidez)
sinasc2.head(2)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,1.0,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,1.0,4,9.0


In [411]:
sinasc2['GESTACAO'] = sinasc2['GESTACAO'].map({'Menos de 22 semanas' : 1, \
'22 a 27 semanas' : 2, '28 a 31 semanas' : 3, '32 a 36 semanas' : 4, '37 a 41 semanas ' : 5, \
'42 semanas e mais' : 6, 'Ignorado' : 9})

In [412]:
moda_gestacao = mode(sinasc2['GESTACAO'])
sinasc2['GESTACAO'] = sinasc2['GESTACAO'].fillna(moda_gestacao)
sinasc2.head(2)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,4.0,1.0,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,4.0,1.0,4,9.0


In [413]:
sinasc2['ESCMAE'] = sinasc2['ESCMAE'].map({'Nenhuma': 1, '1 a 3 anos' : 2, '4 a 7 anos' : 3, \
'8 a 11 anos' : 4, '12 e mais' : 5, 'Ignorado' : 9})

In [414]:
moda_escmae = mode(sinasc2['ESCMAE'])
sinasc2['ESCMAE'] = sinasc2['ESCMAE'].fillna(moda_escmae)
sinasc2.head(2)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,4.0,0.0,4.0,1.0,4,10.0
1,1,29,2.0,4.0,1.0,4.0,1.0,4,9.0


In [415]:
dados = sinasc2[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

for coluna in dados:
    print (f'O Dado {coluna} possui {sinasc2[coluna].isna().sum()} dados faltantes')

O Dado LOCNASC possui 0 dados faltantes
O Dado IDADEMAE possui 0 dados faltantes
O Dado ESTCIVMAE possui 0 dados faltantes
O Dado ESCMAE possui 0 dados faltantes
O Dado QTDFILVIVO possui 0 dados faltantes
O Dado GESTACAO possui 0 dados faltantes
O Dado GRAVIDEZ possui 0 dados faltantes
O Dado CONSULTAS possui 0 dados faltantes
O Dado APGAR5 possui 0 dados faltantes


In [416]:
# 8) seu código aqui
sinasc2.loc[sinasc2['APGAR5'] >= 8, 'APGAR5 CAT'] = 'Normal'
sinasc2.loc[(sinasc2['APGAR5'] >= 6) & (sinasc2['APGAR5'] <= 7), 'APGAR5 CAT'] = 'Asfixia Leve'
sinasc2.loc[(sinasc2['APGAR5'] >= 4) & (sinasc2['APGAR5'] <= 5), 'APGAR5 CAT'] = 'Asfixia Moderada'
sinasc2.loc[(sinasc2['APGAR5'] >= 0) & (sinasc2['APGAR5'] <= 3), 'APGAR5 CAT'] = 'Asfixia Severa'

In [417]:
# 8) seu código aqui
apgar5_porcentagem = sinasc2['APGAR5 CAT'].value_counts(normalize=True)*100
apgar5_porcentagem_formatada = apgar5_porcentagem.map('{:.2f}%'.format)
print (f'A porcentagem de APGAR5 é:\n {apgar5_porcentagem_formatada}')

A porcentagem de APGAR5 é:
 APGAR5 CAT
Normal              98.28%
Asfixia Leve         1.19%
Asfixia Severa       0.27%
Asfixia Moderada     0.25%
Name: proportion, dtype: object


In [418]:
# 9) seu código aqui
sinasc2.columns = sinasc2.columns.str.lower().str.replace(' ', '_', regex=False)
sinasc2.head(2)

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar5_cat
0,1,19,5.0,4.0,0.0,4.0,1.0,4,10.0,Normal
1,1,29,2.0,4.0,1.0,4.0,1.0,4,9.0,Normal
